In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import nltk
from nltk.corpus import stopwords
import string
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.tokenize import sent_tokenize,word_tokenize

In [ ]:
lemmatizer=WordNetLemmatizer()
stops=set(stopwords.words('english'))
puncts=list(string.punctuation)
stops.update(puncts)

In [ ]:
fake_df=pd.read_csv('/kaggle/input/fake-and-real-news-dataset/Fake.csv')
true_df=pd.read_csv('/kaggle/input/fake-and-real-news-dataset/True.csv')

In [ ]:
fake_df['Type']='Fake'
true_df['Type']='True'


In [ ]:
fake_df.info()

In [ ]:
from nltk.corpus import wordnet
def get_simple_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    if tag.startswith('V'):
        return wordnet.VERB
    if tag.startswith('N'):
        return wordnet.NOUN
    if tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [ ]:
def clean_review(words):
    output_words=[]
    words=word_tokenize(words)
    for w in words:
        if w.lower() not in stops:
            pos=pos_tag([w])
            clean_word=lemmatizer.lemmatize(w,pos=get_simple_pos(pos[0][1]))
            output_words.append(clean_word.lower())
    return output_words

In [ ]:
data=pd.concat([fake_df,true_df])

In [ ]:
data = data.sample(frac=1).reset_index(drop=True)


In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le=LabelEncoder()
data['Type_encoded']=le.fit_transform(data['Type'])

In [ ]:
data.columns

In [ ]:
len(data['text'][0])

In [ ]:
from tqdm import tqdm

In [ ]:
text_data=[]
for i in tqdm(range(len(data[:1000]))):
    text_data.append(clean_review(data['title'][i]+data['text'][i]))

In [ ]:
data_proto=data[:1000]
data_proto['processed_text']=text_data
data_proto

In [ ]:
training_documents=data_proto[['Type_encoded','processed_text']][:650]
testing_documents=data_proto[['Type_encoded','processed_text']][650:]

In [ ]:
all_words=[]
for doc in training_documents['processed_text']:
    all_words+=doc

In [ ]:
freq=nltk.FreqDist(all_words)

In [ ]:
common=freq.most_common(1000)
features=[i[0] for i in common]

In [ ]:
features

In [ ]:
def get_feature_dict(words):
    current_features={}
    words_set=set(words)
    for w in features:
        current_features[w] =w in words_set
    return current_features

In [ ]:
training_data=[]
for i in range(len(training_documents)):
    training_data.append((get_feature_dict(training_documents['processed_text'][i]),training_documents['Type_encoded'][i]))

In [ ]:
testing_documents.reset_index(inplace=True,drop=True)

In [ ]:
testing_data=[]
for i in range(len(testing_documents)):
    testing_data.append((get_feature_dict(testing_documents['processed_text'][i]),testing_documents['Type_encoded'][i]))

In [ ]:
from nltk import NaiveBayesClassifier

In [ ]:
classifier=NaiveBayesClassifier.train(training_data)

In [ ]:
print("accuracy for naive bayes with only 1000 data points: ",nltk.classify.accuracy(classifier,testing_data))


In [ ]:
classifier.show_most_informative_features(15)

In [ ]:
from sklearn.svm import SVC
from nltk.classify.scikitlearn import SklearnClassifier

In [ ]:
svc=SVC()
classifier_sklearn=SklearnClassifier(svc)

In [ ]:
classifier_sklearn.train(training_data)

In [ ]:
nltk.classify.accuracy(classifier_sklearn,testing_data)
print("accuracy for SVC with only 1000 data points: ",nltk.classify.accuracy(classifier_sklearn,testing_data))


In [ ]:
w=clean_review(data['text'][1111])
d=get_feature_dict(w)

In [ ]:
classifier.classify(d)

In [ ]:
data.iloc[1111]